# Tâche 3 - Corriger des proverbes avec des *transformers*

Tout comme pour le premier travail pratique, l'objectif de cette tâche est de corriger des proverbes. Cependant vous devrez choisir et utiliser des modèles transformers (2) pour identifier le mot à remplacer et choisir le meilleur mot à insérer en fonction du contexte du proverbe. 


Consignes : 
- Corriger un proverbe consiste à remplacer un verbe par l’un des mots proposés dans la liste. 
- Choix des modèles transformer: Vous devrez choisir 2 modèles, un pour faire l’analyse grammaticale (*POS tagging*) et l'autre pour choisir le mot à insérer dans le proverbe. Deux contraintes : a) pas de LLMs, et b) utilisez des modèles de HuggingFace qui peuvent traiter le français (ou qui sont multilingues). 
- Entraînement des modèles : Vous pouvez utiliser les modèles préentraînés sans modification. Me consulter en cas de doute.  
- Segmentation : Ne pas segmenter un proverbe en sous-phrases. 
- Tokenisation et plongements de mots : Ceux des modèles utilisés. 
- Prétraitement et normalisation : Rien à faire. 
- Code : Utilisez le fichier ***t3_corriger_proverbes.ipynb*** (ce fichier) pour mener vos expérimentations. 
- Données : Évaluez la performance à l’aide du fichier ***data/t3_test_proverbes.json*** et faire l'analyse de vos résultats. 
- Question : Comment se compare l’approche retenue pour cette tâche par rapport à celle du Travail pratique #1?

Vous pouvez ajouter au *notebook* toutes les cellules dont vous avez besoin pour votre code, vos explications ou la présentation de vos résultats. Vous pouvez également ajouter des sous-sections (par ex. des sous-sections 1.1, 1.2 etc.) si cela améliore la lisibilité.

Notes :
- Évitez les bouts de code trop longs ou trop complexes. Par exemple, il est difficile de comprendre 4-5 boucles ou conditions imbriquées. Si c'est le cas, définissez des sous-fonctions pour refactoriser et simplifier votre code. 
- Expliquez sommairement votre démarche.
- Expliquez les choix que vous faites au niveau de la programmation et des modèles (si non trivial).
- Analyser vos résultats. Indiquez ce que vous observez, si c'est bon ou non, si c'est surprenant, etc. 
- Une analyse quantitative et qualitative d'erreurs est intéressante et permet de mieux comprendre le comportement d'un modèle.

## Section 1 - Lecture du fichier de test 

Le fichier de test ***./data/t3_test_proverbes.json*** contient les proverbes modifiés, la liste de mots candidats et la bonne version du proverbe. 

In [2]:
import json

def load_tests(filename):
    with open(filename, 'r', encoding='utf-8') as fp:
        test_data = json.load(fp)
    return test_data

In [3]:
test_fn = './data/t3_test_proverbes.json'  # Le fichier de test = À modifier selon votre configuration

In [4]:
tests = load_tests(test_fn)

In [5]:
import pandas as pd

def get_dataframe(test_proverbs):
    return pd.DataFrame.from_dict(test_proverbs, orient='columns', dtype=None, columns=None)

df = get_dataframe(tests)
df

,Masked,Word_list,Proverb
0,a beau mentir qui part de loin,"[vient, revient]",a beau mentir qui vient de loin
1,a beau dormir qui vient de loin,"[partir, mentir]",a beau mentir qui vient de loin
2,l’occasion forge le larron,"[fait, occasion]",l’occasion fait le larron
3,"endors-toi, le ciel t’aidera","[bouge, aide]","aide-toi, le ciel t’aidera"
4,"aide-toi, le ciel t’aura","[aidera, aide]","aide-toi, le ciel t’aidera"
5,"ce que femme dit, dieu le veut","[dit, veut]","ce que femme veut, dieu le veut"
6,"ce que femme veut, dieu le souhaite","[dit, veut]","ce que femme veut, dieu le veut"
7,bien mal acquis ne sait jamais,"[profite, fait]",bien mal acquis ne profite jamais
8,bon ouvrier ne déplace pas ses outils,"[fait, querelle]",bon ouvrier ne querelle pas ses outils
9,"pour le fou, c’était tous les jours fête","[est, es]","pour le fou, c’est tous les jours fête"


## Section 2 - Code pour repérer les mots qui pourraient être remplacés dans un proverbe modifié

Expliquez ici comment vous procédez pour identifier les mots d'un proverbe qui pourraient faire l'objet d'une substitution.  



## Section 3 - Modèle et code pour corriger un proverbe

Expliquez ici comment vous procédez pour choisir la meilleure version parmi les proverbes modifiés. 

## Section 4 - Expérimentations et analyse de résultats
